In [ ]:
# step 9
from mne.time_frequency import tfr_morlet
from alive_progress import alive_bar
import matplotlib.pyplot as plt
from tqdm import tqdm
import nibabel as nib
import pandas as pd
import numpy as np
import seaborn as sns
import scipy
import h5py
import mne
import sys
import os
# 输入的是
subject_num = 34
behaviorVar_exp4_loc = 'E:/workspace/Trust_Data_and_Results/Haoming/varibility/behavior_result/EXP4.txt'
result_loc = 'E:/workspace/Trust_Data_and_Results/Haoming/varibility/behavior_result/'
eegVar_loc = 'E:/workspace/Trust_Data_and_Results/Haoming/varibility/result/varibilit_corr_20/all_var_data_one_all_6band.npy'
var_loc = 'E:/workspace/Trust_Data_and_Results/Haoming/varibility/result/varibilit_corr_20/'

behaviorVar_exp4_all_data = []

experiment_4_event = [30, 31, 32, 33]

event_name_exp2 = ['0Trust', '1Trust', '2Trust', '3Trust']

# 时间窗的长度， 单位
window_length = 50

# 需要分析的数据起止时间 第一位是开始的时间，第二位是结束的时间
involve_time = [-0.1, 0.7]
# 34 1 1 63

In [ ]:
#从text中提取数据
all_subject_list = pd.read_csv(behaviorVar_exp4_loc, delimiter='\t')#, names=['Subject', 'Age'	,'Sex',	'face1.ACC', 'face1.RESP', 'face1.RT', 'or'/'type'])

# 所有被试的编号
#all_subject_number = np.sort(np.unique(all_subject_list[all_subject_list.keys()[0]]))

# 减去空集 NAN
all_subject_list = all_subject_list.dropna()

all_subject_number = np.sort(np.unique(all_subject_list[all_subject_list.keys()[0]]))

In [ ]:
# 把所有被试的平均数据放在以
all_subject, all_subject_trialnum = [], []
event_value = {}

for iter_subject in range(len(all_subject_number)):

    # 先获得每一个被试的数据位置（行数）
    iter_subject_position = np.where(all_subject_list[all_subject_list.keys()[0]] == all_subject_number[iter_subject])

    # total_len.append(len(iter_subject_position[0]))
    iter_subject_list = all_subject_list.iloc[iter_subject_position]
    
    # 提取四种条件，其平均数 以及该条件的个数
    iter_subject, ter_subiject_trialnum = [], []
    
    # 对每一个event进行一次循环（目前共四个）
    for iter_event in experiment_4_event:
        # 检查每一个event在该被试中对应的位置
        iter_event_position = np.where(iter_subject_list[iter_subject_list.keys()[-1]] == iter_event)
        # 统计每个event的数量
        event_num = len(iter_event_position[0])
        # 截取出该event的dataframe
        iter_event_list = iter_subject_list.iloc[iter_event_position]
        # 截取数据段
        iter_event_value = np.array(iter_event_list[iter_event_list.keys()[5]])
        # 求平均数
        # iter_event_value = np.mean(iter_event_list[iter_event_list.keys()[3]])
        # 存到数组的第一个维度
        iter_subject.append(iter_event_value)
        # ter_subiject_trialnum.append(event_num)
    #event_value = pd.DataFrame(event_value)
    # 存到总数组的第二个维度
    all_subject.append(iter_subject)
    # all_subject_trialnum.append(ter_subiject_trialnum)


# 储存该np数组
all_subject = np.array(all_subject)
all_subject = all_subject - 1 
# all_subject_trialnum = np.array(all_subject_trialnum)

if not os.path.exists(result_loc):
    os.makedirs(result_loc)

np.save(result_loc + 'all_subject_exp4.npy', all_subject)
# np.save(result_loc + 'all_subject_trialnum.npy', all_subject_trialnum)
#print(total_len)

In [ ]:
import numpy as np
all_subject_exp4 = np.load(result_loc + 'all_subject_exp4.npy')
all_subject_exp4_condition01 = np.mean(all_subject_exp4[:,0:2,:], axis = 1)
all_subject_exp4_condition23 = np.mean(all_subject_exp4[:,2:4,:], axis = 1)
all_subject_exp4_condition01 = np.mean(all_subject_exp4_condition01, axis = 1)
all_subject_exp4_condition23 = np.mean(all_subject_exp4_condition23, axis = 1)


In [ ]:
exp_mean_deference = all_subject_exp4_condition23 - all_subject_exp4_condition01
behavior_varibility = exp_mean_deference

eeg_varibility = np.load(var_loc + 'all_var_data_nor.npy', allow_pickle=True).item()
print(behavior_varibility.shape)
print(eeg_varibility.keys())

print(eeg_varibility['data'].shape)
print(eeg_varibility['event_name'])
# print(behavior_varibility)

# Subjects with incomplete data were removed
bad_subject = np.array([3, 5, 15]) -1
all_subject = np.arange(0,34,1)

# Delete specific elements
good_subject = np.setdiff1d(all_subject, bad_subject)


eeg_varibility['data'] = eeg_varibility['data'][good_subject]
behavior_varibility = behavior_varibility[good_subject]
print(eeg_varibility['data'].shape)
print(behavior_varibility.shape)

In [ ]:
channel_location_dict = {"Fp1":	[-26.25, 83, -18.25],
"Fp2": [26.25, 83, -18.25],
"F3": [-48.75, 57.5, 36.5],
"F4": [48.75, 57.5, 36.5],
"C3": [-63.75, -13, 65.75],
"C4": [63.75, -13, 65.75],
"P3": [-48, -86.5, 53],
"P4": [48, -86.5, 53],
"O1": [-27, -118, 4.25],
"O2": [27, -118, 4.25],
"F7": [-69, 44, -18.25],
"F8": [69, 44, -18.25],
"T7": [-84, -18.25, -13],
"T8": [84, -18.25, -13],
"P7": [-69, -80.5, -4.75],
"P8": [69, -80.5, -4.75],
"Fz": [0, 63.5, 59],
"Cz": [0, -10, 99.5],
"Pz": [0, -88, 77],
"FC1": [-32.25, 28.25, 77.75],
"FC2": [32.25, 28.25, 77.75],
"CP1": [-35.25, -52, 90.5],
"CP2": [35.25, -52, 90.5],
"FC5": [-75.75, 20, 21.5],
"FC6": [75.75, 20, 21.5],
"CP5": [-78, -51.25, 31.25],
"CP6": [78, -51.25, 31.25],
"FT9": [-72.75, 8.75, -55.75],
"FT10": [72.75, 8.75, -55.75],
"TP9": [-71.25, -49, -49.75],
"TP10": [71.25, -49, -49.75],
"F1": [-25.5, 62, 53.75],
"F2": [25.5, 62, 53.75],
"C1": [-35.25, -11.5, 89.75],
"C2": [35.25, -11.5, 89.75],
"P1": [-27, -88, 70.25],
"P2": [27, -88, 70.25],
"AF3": [-32.25, 80.75, 11.75],
"AF4": [32.25, 80.75, 11.75],
"FC3": [-59.25, 25.25, 54.5],
"FC4": [59.25, 25.25, 54.5],
"CP3": [-63, -52, 66.5],
"CP4": [63, -52, 66.5],
"PO3": [-31.5, -109.75, 32.75],
"PO4": [31.5, -109.75, 32.75],
"F5": [-63, 51.5, 11.75],
"F6": [63, 51.5, 11.75],
"C5": [-81.75, -14.5, 29],
"C6": [81.75, -14.5, 29],
"P5": [-63.75, -83.5, 26.75],
"P6": [63.75, -83.5, 26.75],
"AF7": [-50.25, 68, -19],
"AF8": [50.25, 68, -19],
"FT7": [-79.5, 15.5, -16],
"FT8": [79.5, 15.5, -16],
"TP7": [-80.25, -49.75, -8.5],
"TP8": [80.25, -49.75, -8.5],
"PO7": [-50.25, -103, 0.5],
"PO8": [50.25, -103, 0.5],
"Fpz": [0, 83, -18.25],
"CPz": [0, -53.5, 98],
"POz": [0, -109, 44],
"Oz": [0, -122.5, 8]}
# ['Fp1', 'Fp2', 'Fz', 'Cz', 'Pz', 'Fpz', 'CPz', 'POz', 'Oz'].
mymontage = mne.channels.make_dig_montage(ch_pos=channel_location_dict)#, nasion=[0, -10, 99.5], lpa=[-71.25, -49, -49.75], 
#                                        rpa=[71.25, -49, -49.75])

In [ ]:

power_name = ['alpha']


time_series = np.arange(-0.2, 1, 0.001)
time_series = np.around(time_series,3)
start_time = np.where(time_series == involve_time[0])[0][0]
end_time = np.where(time_series == involve_time[1])[0][0]

iter_power = 2
figure_result_loc = 'E:/workspace/Trust_Data_and_Results/Haoming/varibility/result/varibilit_corr_20/corr_result/temporal_figure/exp1eeg_exp4beha_spearman'

iter_condition_and_power_result = []
for iter_elec in tqdm(range(eeg_varibility['data'].shape[-2])):
    
    iter_elec_result = []
    
    for iter_window in range(end_time - start_time):# 步长为1
        
        t_begin = start_time + iter_window - round(window_length/2)
        t_end = start_time + iter_window + round(window_length/2)

        iter_eeg_varibility_data = eeg_varibility['data'][:,iter_power+1, 0, iter_elec, t_begin:t_end]
        iter_eeg_varibility_data_mean = np.mean(iter_eeg_varibility_data, axis=-1)

        
        x = behavior_varibility
        y = iter_eeg_varibility_data_mean

        
        scipy_corr = scipy.stats.pearsonr(x, y)
        

        if (scipy_corr[1] <= 0.05) & (scipy_corr[0] > 0):
            iter_elec_result.append(1)
        elif (scipy_corr[1] <= 0.05) & (scipy_corr[0] < 0):
            iter_elec_result.append(-1)
        else:
            iter_elec_result.append(0)

    
    iter_condition_and_power_result.append(iter_elec_result)

iter_condition_and_power_result= np.array(iter_condition_and_power_result)
# create a new epochs info
info = mne.create_info(
    ch_names = eeg_varibility['ch_names'],
    ch_types = 'eeg',
    sfreq = 1000)

# create a new ROI based epochs
significiant_evoked = mne.EvokedArray(data = iter_condition_and_power_result, 
    info = info, tmin=-0.1)

significiant_evoked.set_montage(mymontage)
# plt.close('all')
#plt.rcParams['figure.figsize'] = (20,20)
all_time = np.arange(-0.1, 0.35, 0.002)
all_time = np.around(all_time, 3)
for iter_times in range(len(all_time)):

    times = all_time[iter_times]
    fig = significiant_evoked.plot_topomap(outlines='head',times=times, size=5, vmin=-1500000, vmax=1500000, show_names=False, sensors='kv', extrapolate='head', cmap='RdBu_r', scalings=None, colorbar=False)# cmap GnBu


    iter_figure_result_loc = figure_result_loc + '/' + power_name[iter_power] + '/'
    if not os.path.exists(iter_figure_result_loc):
        os.makedirs(iter_figure_result_loc)

    fig.savefig(iter_figure_result_loc + str(iter_times) + '_' + str(times) + '_pic.png')
